Before running the file Upload all your data set on your goole drive in a zip format

In [ ]:
#Mount our google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#before running this please change the RUNTIME to GPU (Runtime -> Change runtime type -> set harware accelarotor as GPU)
#download and unzip the data from google drive Colab environment
from google_drive_downloader import GoogleDriveDownloader as gdd
#use only file id of the link
#Note: Below link is just an example, Not an actual link. Actual Links are in ReadMe file
#https://drive.google.com/file/d/1ubvKLzBDe5i1acxgGUK6ObeNBYCKUS07/view?usp=sharing
#https://drive.google.com/drive/folders/1LVDiZ_AmHUyRxmUIWOI7eyaMDKMszEdn?usp=sharing
url = '1LVDiZ_AmHUyRxmUIWOI7eyaMDKMszEdn'
gdd.download_file_from_google_drive(file_id = url,dest_path='./data.zip',unzip=True)

Unzipping...

/usr/local/lib/python3.9/dist-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "1LVDiZ_AmHUyRxmUIWOI7eyaMDKMszEdn" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


In [ ]:
#To get the average frame count 
import json
import glob
import numpy as np
import cv2
import copy
#change the path accordingly
video_files =  glob.glob('/content/drive/MyDrive/dataset/dfdc_train_part_0/*.mp4')
#video_files1=  glob.glob('/content/drive/MyDrive/dataset/dfdc_train_part_1/*.mp4')
#video_files2=  glob.glob('/content/drive/MyDrive/dataset/dfdc_train_part_2/*.mp4')
#video_files3=  glob.glob('/content/drive/MyDrive/dataset/dfdc_train_part_3/*.mp4')
#video_files4=  glob.glob('/content/drive/MyDrive/dataset/dfdc_train_part_4/*.mp4')
#video_files5=  glob.glob('/content/drive/MyDrive/dataset/dfdc_train_part_5/*.mp4')
#video_files6=  glob.glob('/content/drive/MyDrive/dataset/dfdc_train_part_6/*.mp4')
#video_files7=  glob.glob('/content/drive/MyDrive/dataset/dfdc_train_part_7/*.mp4')
#video_files = video_files + video_files1
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames [300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 30

In [ ]:
# to extract frame
def frame_extract(path):
  vidObj = cv2.VideoCapture(path) 
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image
!pip3 install face_recognition
!mkdir '/content/drive/My Drive/Face_only_data1'
#import torch
#import torchvision
#from torchvision import transforms
#from torch.utils.data import DataLoader
#from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm
# process the frames
def create_face_videos(path_list,out_dir):
  already_present_count =  glob.glob(out_dir+'*.mp4')
  print("No of videos already present " , len(already_present_count))
  for path in tqdm(path_list):
    out_path = os.path.join(out_dir,path.split('/')[-1])
    file_exists = glob.glob(out_path)
    if(len(file_exists) != 0):
      print("File Already exists: " , out_path)
      continue
    frames = []
    flag = 0
    face_all = []
    frames1 = []
    out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
    for idx,frame in enumerate(frame_extract(path)):
      #if(idx % 3 == 0):
      if(idx <= 150):
        frames.append(frame)
        if(len(frames) == 4):
          faces = face_recognition.batch_face_locations(frames)
          for i,face in enumerate(faces):
            if(len(face) != 0):
              top,right,bottom,left = face[0]
            try:
              out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
            except:
              pass
          frames = []
    try:
      del top,right,bottom,left
    except:
      pass
    out.release()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/content/drive/My Drive/Face_only_data1’: File exists


In [ ]:
create_face_videos(video_files,'/content/drive/My Drive/Face_only_data1/')

No of videos already present  1262


  0%|          | 0/1412 [00:00<?, ?it/s]

File Already exists:  /content/drive/My Drive/Face_only_data1/sijzcbwppo.mp4
File Already exists:  /content/drive/My Drive/Face_only_data1/qcjwsygnxe.mp4
File Already exists:  /content/drive/My Drive/Face_only_data1/nwtdlrhqtf.mp4
File Already exists:  /content/drive/My Drive/Face_only_data1/jquevmhdvc.mp4
File Already exists:  /content/drive/My Drive/Face_only_data1/yllztsrwjw.mp4
File Already exists:  /content/drive/My Drive/Face_only_data1/rmuxlgsedw.mp4
File Already exists:  /content/drive/My Drive/Face_only_data1/wsakxxhoae.mp4
File Already exists:  /content/drive/My Drive/Face_only_data1/ludeqgyyct.mp4
File Already exists:  /content/drive/My Drive/Face_only_data1/jinomojicv.mp4
File Already exists:  /content/drive/My Drive/Face_only_data1/mvnahpueuk.mp4
File Already exists:  /content/drive/My Drive/Face_only_data1/ridrlfphki.mp4
File Already exists:  /content/drive/My Drive/Face_only_data1/rrdamrrple.mp4
File Already exists:  /content/drive/My Drive/Face_only_data1/tbujzmjpsl.mp4